In [ ]:
#importing library
import warnings
warnings. simplefilter(action='ignore', category=Warning)
import pandas as pd
import numpy as np
import ffn
from tabulate import tabulate

In [ ]:
import datetime as dt

In [ ]:
# importing file from a local folder
from google.colab import files
uploaded = files.upload()

Saving tradelog.csv to tradelog.csv


In [ ]:
df=pd.read_csv("tradelog.csv")

In [ ]:
#set date
startdate=datetime.datetime(2020,7,8)
enddate=datetime.datetime(2020,12,31)
num_days=(enddate-startdate)
year=num_days/365
starting_balance=6500

In [ ]:
df['PNL']=(df['Exit Price']-df['Entry Price'])
df['ROR']=df['PNL']/df['Entry Price']
df['Balance']=round(starting_balance + df.ROR.cumsum(),2)

In [ ]:
df.head(180)
df.dtypes

Unnamed: 0       int64
Ticker          object
Entry Time      object
Entry Price    float64
Exit Time       object
Exit Price     float64
PNL            float64
ROR            float64
Balance        float64
dtype: object

In [ ]:
df.head()

,Unnamed: 0,Ticker,Entry Time,Entry Price,Exit Time,Exit Price,PNL,ROR,Balance
0,1,BANKNIFTY09JUL2022800PE,08-07-2020 09:29,163.50,08-07-2020 13:15,301.00,137.50,1.840979,6501.84
1,2,BANKNIFTY09JUL2022700CE,09-07-2020 09:29,162.50,09-07-2020 10:34,166.75,4.25,1.026154,6501.89
2,3,BANKNIFTY09JUL2022700PE,09-07-2020 09:29,110.55,09-07-2020 10:34,147.00,36.45,1.329715,6502.51
3,4,BANKNIFTY16JUL2022700CE,10-07-2020 09:29,295.10,10-07-2020 15:24,437.55,142.45,1.482718,6503.72
4,5,BANKNIFTY16JUL2022700PE,10-07-2020 09:29,555.20,10-07-2020 15:24,403.15,-152.05,0.726135,6502.70


In [ ]:
#creating new columns
df['Entry Time']=pd.to_datetime(df['Entry Time'])
df['Exit Time']=pd.to_datetime(df['Exit Time'])

In [ ]:
df.dtypes

Unnamed: 0              int64
Ticker                 object
Entry Time     datetime64[ns]
Entry Price           float64
Exit Price            float64
PNL                   float64
ROR                   float64
Balance               float64
dtype: object

In [ ]:
Parameters = [ 'Total Trade',  'Profitable trades', 'Loss making trade', 'Win Rate','Total Profit', 'Total Loss', 'Average Loss per Trade', 'Average Profit per Trade', 'Risk Reward', 'Expectancy','Avg_ror']
#total_traded_scripts = len(df['Ticker'].unique())
total_trade = len(df.index)
Profitable_trades = len(df[df['PNL']>0])
Loss_making_trades = len(df[df['PNL']<0])
Win_rate = round((Profitable_trades/total_trade), 2)
total_profit = round(df[df.PNL > 0].PNL.sum(), 2)
total_loss  = round(df[df.PNL <= 0].PNL.sum(), 2)
Average_loss_per_trade = round(total_loss/Loss_making_trades, 2)
Average_profit_per_trade = round(total_profit/Profitable_trades, 2)
Risk_reward = f'1:{ round(Average_profit_per_trade/Average_loss_per_trade, 2)}'
Expectancy = (Win_rate * Average_profit_per_trade)-((1- Win_rate) * Average_loss_per_trade)
Avg_ror=str(round(df['ROR'].mean(),2))+'%'
risk_free_intrest=.05
data_points = [total_trade,Profitable_trades,Loss_making_trades, Win_rate, total_profit, total_loss, Average_loss_per_trade, Average_profit_per_trade, Risk_reward, Expectancy,Avg_ror]
data = list(zip(Parameters,data_points ))
print(tabulate(data, ['Parameters', 'Values']))


Parameters                Values
------------------------  --------
Total Trade               249
Profitable trades         153
Loss making trade         96
Win Rate                  0.61
Total Profit              11642.6
Total Loss                -10132.2
Average Loss per Trade    -105.54
Average Profit per Trade  76.1
Risk Reward               1:-0.72
Expectancy                87.5816
Avg_ror                   0.17%


In [ ]:
#changing index to calculate other details.
df.index
df.set_index('Exit Time',inplace=True)

In [ ]:
df.index

DatetimeIndex(['2020-08-07 13:15:00', '2020-09-07 10:34:00',
               '2020-09-07 10:34:00', '2020-10-07 15:24:00',
               '2020-10-07 15:24:00', '2020-07-13 11:11:00',
               '2020-07-13 11:11:00', '2020-07-14 10:56:00',
               '2020-07-14 10:56:00', '2020-07-15 12:03:00',
               ...
               '2020-12-24 10:15:00', '2020-12-24 10:15:00',
               '2020-12-28 13:43:00', '2020-12-28 13:43:00',
               '2020-12-29 13:10:00', '2020-12-29 13:10:00',
               '2020-12-30 10:24:00', '2020-12-30 10:24:00',
               '2020-12-31 10:37:00', '2020-12-31 10:37:00'],
              dtype='datetime64[ns]', name='Exit Time', length=249, freq=None)

In [ ]:
perm=ffn.core.PerformanceStats(df['Balance'],rf=.05)

In [ ]:
#here we get all statics related details of our data
perm.display()

Stats for Balance from 2020-08-07 13:15:00 - 2020-12-31 10:37:00
Annual risk-free rate considered: 5.00%
Summary:
Total Return      Sharpe  CAGR    Max Drawdown
--------------  --------  ------  --------------
0.63%              -1.59  0.46%   -0.55%

Annualized Returns:
mtd    3m     6m     ytd    1y    3y    5y    10y    incep.
-----  -----  -----  -----  ----  ----  ----  -----  --------
0.11%  0.33%  0.19%  0.45%  -     -     -     -      0.46%

Periodic:
        daily    monthly    yearly
------  -------  ---------  --------
sharpe  -1.59    -9.19      -
mean    0.96%    0.11%      -
vol     2.46%    0.52%      -
skew    -1.44    -1.35      -
kurt    2.45     3.32       -
best    0.24%    0.23%      -
worst   -0.54%   -0.36%     -

Drawdowns:
max     avg       # days
------  ------  --------
-0.55%  -0.42%      22.9

Misc:
---------------  ------
avg. up month    0.07%
avg. down month  -0.16%
up year %        -
12m up %         -
---------------  ------
